# Data and history

In [14]:
import pytse_client as tse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tickers = tse.download(symbols="خودرو", adjust=True, write_to_csv=True)
df = pd.DataFrame.from_dict(tickers["خودرو"])
df["label"] = ''
df.head()

,date,open,high,low,adjClose,value,volume,count,yesterday,close,label
0,2001-03-25,54.0,54.0,54.0,54.0,310368080,110870,115,54.0,54.0,
1,2001-03-26,54.0,55.0,54.0,54.0,270595190,96613,98,54.0,54.0,
2,2001-03-27,54.0,54.0,54.0,54.0,466405131,166600,120,54.0,54.0,
3,2001-03-28,54.0,55.0,54.0,55.0,227566666,80676,104,54.0,55.0,
4,2001-03-31,55.0,55.0,54.0,55.0,500068371,177362,161,55.0,55.0,


In [15]:
#Labeling

ts = df[["adjClose"]]
k = 30
max = ts[::-1].rolling(k,1).max().shift(1)[::-1]
min = ts[::-1].rolling(k,1).min().shift(1)[::-1]
ts = pd.concat([ts, max, min], axis = 1) 

#filling the label column
for i, row in ts.iterrows():
    if (ts.iloc[i,0] == ts.iloc[i,1]):
        df.iloc[i,10] = -1
    elif (ts.iloc[i,0] == ts.iloc[i,2]):
        df.iloc[i,10] = 1
    else:
        df.iloc[i,10] = 0


#df['label'].value_counts()[0]
df.head()

,date,open,high,low,adjClose,value,volume,count,yesterday,close,label
0,2001-03-25,54.0,54.0,54.0,54.0,310368080,110870,115,54.0,54.0,1
1,2001-03-26,54.0,55.0,54.0,54.0,270595190,96613,98,54.0,54.0,1
2,2001-03-27,54.0,54.0,54.0,54.0,466405131,166600,120,54.0,54.0,0
3,2001-03-28,54.0,55.0,54.0,55.0,227566666,80676,104,54.0,55.0,1
4,2001-03-31,55.0,55.0,54.0,55.0,500068371,177362,161,55.0,55.0,0


# Random Forest Classification

In [16]:
X = df.iloc[:, 1:10].values
y = df.iloc[:, -1].values

y=y.astype('int')

In [17]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [18]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
#Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [20]:
#Predicting the Test set results

from sklearn.metrics import confusion_matrix

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 54  70  11]
 [ 34 781  25]
 [ 11  69  62]]
